In [ ]:
pip install pandas

In [ ]:
pip install matplotlib seaborn

# CIC-IoMT- 2024 veri seti için Keşifsel Veri Analizi (EDA) yapmanız, görselleştirmeleri yorumlamanız beklenmektedir. Literatür taraması yaparak yorumlarınızı güçlendiriniz. 

## 1- Veriyi Anlama ve Hazırlama (Data Understanding & Preparation)

    1- Veri setini yükleyin ve ilk gözlemleri yapın (head(), info(), describe() vb.).

    2- Veri setindeki sütunların (değişkenlerin) ne anlama geldiğini araştırın ve açıklayın.

    3- Eksik verileri (NaN değerler) tespit edin. Bu eksik verilerle nasıl başa çıktığınızı (siliyor musunuz, dolduruyor musunuz, neden?) gerekçeleriyle açıklayın.

    4- Veri tiplerini kontrol edin ve anlamsal olarak hatalı olanları (örneğin sayısal olması gereken bir sütunun object olması) düzeltin.

    5- Varsa aykırı değerleri (outliers) tespit edin. Bu değerleri nasıl ele aldığınızı ve neden bu yöntemi seçtiğinizi belirtin.


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("WiFI_and_MQTT/attacks/CSV/test/TCP_IP-DDoS-UDP1_test.pcap.csv")

# 1- Veri setini yükleyin ve ilk gözlemleri yapın (head(), info(), describe() vb.).
# 2- Veri setindeki sütunların (değişkenlerin) ne anlama geldiğini araştırın ve açıklayın.

print(df.head())
# dataset'in 'lk 5 satirini getirir

print("-"*100)

print(df.info())
# dataset'deki feature'lari gosterir

print("-"*100)

print(df.describe())
# dataset'i tanimla

In [ ]:
# 3- Eksik verileri (NaN değerler) tespit edin. Bu eksik verilerle nasıl başa çıktığınızı (siliyor musunuz, dolduruyor musunuz, 
#    neden?) gerekçeleriyle açıklayın.

na_counts = df.isna().sum().sort_values(ascending=False)
na_ratio = (df.isna().mean()*100).sort_values(ascending=False)

total_na = na_counts.sum()

print(f"🔹 Toplam NaN değerler: {total_na}")
for col, count in na_counts.items():
    print(f"{col}: {count}")

print("-" * 100)

num_cols = df.select_dtypes(include=[np.number]).columns  # sadece sayısal sütunlar
inf_mask = np.isinf(df[num_cols])
inf_counts = inf_mask.sum().sort_values(ascending=False)
total_inf = inf_counts.sum()

print(f"🔹 Toplam ∞ (inf) değerler: {total_inf}")
for col, count in inf_counts.items():
    print(f"{col}: {count}")

In [ ]:
import pandas as pd
import numpy as np

num_cols = df.select_dtypes(include=[np.number]).columns

df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan)

fill_report = []

for col in num_cols:
    before = df[col].isna().sum()
    if before > 0:
        mean_value = df[col].mean()
        df[col] = df[col].fillna(mean_value)
        fill_report.append({
            "Feature": col,
            "Doldurulan NaN Sayısı": before,
            "Kullanılan Ortalama": round(mean_value, 6)
        })

if fill_report:
    fill_df = pd.DataFrame(fill_report)
    print("Ortalama ile Doldurulan Sütunlar:")
    print(fill_df.to_string(index=False))
else:
    print("Doldurulacak NaN veya ∞ değer bulunamadı.")

print("\nSon Kontrol:")
print(f"Kalan NaN toplamı: {df.isna().sum().sum()}")
print(f"Kalan ∞ toplamı: {np.isinf(df[num_cols]).sum().sum()}")


In [ ]:
# 4- Veri tiplerini kontrol edin ve anlamsal olarak hatalı olanları (örneğin sayısal olması gereken bir sütunun object olması) düzeltin.
print(df.dtypes)

In [ ]:
# 5- Varsa aykırı değerleri (outliers) tespit edin. Bu değerleri nasıl ele aldığınızı ve neden bu yöntemi seçtiğinizi belirtin.

import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x=df["Rate"])
plt.show()

In [ ]:
Q1 = df["Rate"].quantile(0.25)
Q3 = df["Rate"].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers = df[(df["Rate"] < lower) | (df["Rate"] > upper)]
print("Aykırı değer sayısı:", len(outliers))


## 2- Tek Değişkenli Analiz (Univariate Analysis)

1- Sayısal değişkenlerin dağılımını histogram ve kutu grafiği (box plot) gibi grafiklerle görselleştirin. Her bir grafik için gözlemlerinizi (dağılımın çarpıklığı, merkezi eğilim vb.) yazın.

2- Kategorik değişkenlerin frekanslarını bar grafikleri ile görselleştirin.


In [ ]:
# 1- Sayısal değişkenlerin dağılımını histogram ve kutu grafiği (box plot) gibi grafiklerle görselleştirin. Her bir grafik için 
# gözlemlerinizi (dağılımın çarpıklığı, merkezi eğilim vb.) yazın.

import matplotlib.pyplot as plt
import seaborn as sns

num_vars = ["Rate", "Time_To_Live", "Header_Length"]

for col in num_vars:
    plt.figure(figsize=(12,4))
    
    # Histogram
    plt.subplot(1,2,1)
    sns.histplot(df[col], bins=50, kde=True)
    plt.title(f"{col} - Histogram")
    
    # Boxplot
    plt.subplot(1,2,2)
    sns.boxplot(x=df[col])
    plt.title(f"{col} - Boxplot")
    
    plt.show()

In [ ]:
# 2- Kategorik değişkenlerin frekanslarını bar grafikleri ile görselleştirin.

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches

plt.figure(figsize=(8,5))
sns.countplot(x="Protocol Type", data=df)
plt.title("Protocol Type - Frekans Dağılımı")
plt.xlabel("Protocol Type")
plt.ylabel("Count")

# Legend için sadece yazılar
protocol_labels = [
    "0 → HOPOPT",
    "1 → ICMP",
    "6 → TCP",
    "17 → UDP"
]

# Sağ üst köşeye sadece yazılardan oluşan kutu ekle
plt.legend(labels=protocol_labels, title="Protocol Kodları", loc="upper right",
           frameon=True, fontsize=9, title_fontsize=10)

plt.show()

## 3- Çok Değişkenli Analiz (Bivariate/Multivariate Analysis)
    1- Değişkenler arasındaki ilişkileri keşfedin.
    2- Korelasyon Analizi: Sayısal değişkenler arasındaki korelasyon matrisini oluşturun ve bir ısı haritası (heatmap) ile görselleştirin. Güçlü pozitif veya negatif ilişkileri yorumlayın.
    3- Saçılım Grafikleri (Scatter Plots): Özellikle ilginç bulduğunuz değişken çiftleri arasındaki ilişkiyi saçılım grafikleriyle inceleyin.
    4- Karşılaştırmalı Analiz: Kategorik ve sayısal değişkenleri bir arada analiz edin.

In [ ]:
# 1- Değişkenler arasındaki ilişkileri keşfedin.
# 2- Korelasyon Analizi: Sayısal değişkenler arasındaki korelasyon matrisini oluşturun ve bir ısı haritası (heatmap) ile görselleştirin. 
#    Güçlü pozitif veya negatif ilişkileri yorumlayın.

import seaborn as sns
import matplotlib.pyplot as plt

corr = df.corr()

plt.figure(figsize=(12,8))
sns.heatmap(corr, cmap="coolwarm", annot=False, linewidths=0.5)
plt.title("Korelasyon Matrisi - Heatmap")
plt.show()

In [ ]:
# 3- Saçılım Grafikleri (Scatter Plots): Özellikle ilginç bulduğunuz değişken çiftleri arasındaki ilişkiyi saçılım grafikleriyle inceleyin.

sns.scatterplot(x="Rate", y="Time_To_Live", data=df, alpha=0.5)
plt.title("Rate vs TTL")
plt.show()

sns.scatterplot(x="Rate", y="Tot size", data=df, alpha=0.5)
plt.title("Rate vs Tot Size")
plt.show()

In [ ]:
# 4- Karşılaştırmalı Analiz: Kategorik ve sayısal değişkenleri bir arada analiz edin.

files = {
    "Benign": "WiFI_and_MQTT/attacks/CSV/test/Benign_test.pcap.csv",
    "ARP_Spoofing": "WiFI_and_MQTT/attacks/CSV/test/ARP_Spoofing_test.pcap.csv",
    "MQTT-DoS-Connect_Flood": "WiFI_and_MQTT/attacks/CSV/test/MQTT-DoS-Connect_Flood_test.pcap.csv"
}

frames = []
for label, path in files.items():
    temp = pd.read_csv(path)
    temp["Label"] = label  # saldırı türünü kategori olarak ekle
    # inf -> NaN
    num_cols = temp.select_dtypes(include=[np.number]).columns
    temp[num_cols] = temp[num_cols].replace([np.inf, -np.inf], np.nan)
    # NaN doldurma (ör: Rate için medyan)
    if "Rate" in temp.columns:
        temp["Rate"] = temp["Rate"].fillna(temp["Rate"].median())
    frames.append(temp)

# Tümünü tek DataFrame'e birleştir
df_all = pd.concat(frames, ignore_index=True)

print(df_all["Label"].value_counts())
plt.figure(figsize=(10,6))
sns.boxplot(x="Label", y="Rate", data=df_all)  # df_all = birden fazla dosyayı birleştirdiğimiz tablo
plt.xticks(rotation=45)
plt.title("Saldırı Türüne Göre Rate Dağılımı")
plt.show()